In [117]:
import json
import sqlite3
import os 
import pandas as pd

FROM_DIR = 'source'

conn = sqlite3.Connection('data.db')
cur = conn.cursor()

cur.execute(f"""
    select e.id, vector, url, title
    from embeddings e
    join (
        select * 
        from articles
        where lower(title) like 'altruism'
        limit 1
    ) a
    using(id)
""")
cur.fetchone()

(336,
 '[0.013182079419493675, -0.024738267064094543, 0.02943071722984314, 0.09730659425258636, 0.027207976207137108, -0.023709220811724663, -0.019551875069737434, 0.0346376895904541, -0.0073885503225028515, 0.02100282907485962, -0.0012374278157949448, -0.04556615650653839, -0.008854941464960575, -0.035604991018772125, 0.004604980815201998, 0.0310977715998888, 0.035769641399383545, -0.015126976184546947, 0.030192211270332336, 0.0015705814585089684, 0.035440344363451004, 0.00900929793715477, -0.03311470150947571, 0.024182582274079323, -0.02928665094077587, -0.010300750844180584, 0.0016516188625246286, 0.06034325808286667, 0.021918680518865585, -0.03850690275430679, 0.025911379605531693, -0.0236063152551651, 0.011473863385617733, -0.013634859584271908, 0.011370958760380745, 0.04778889939188957, -0.002079959260299802, -0.057791225612163544, 0.014272868633270264, -0.011556186713278294, -0.03840399906039238, -0.04132648929953575, 0.003118009539321065, 0.028998516499996185, 0.003437013830989

In [119]:
import json
import sqlite3
import os 
import pandas as pd
from scipy.spatial import distance

FROM_DIR = 'source'

conn = sqlite3.Connection('data.db')
cur = conn.cursor()

search = "kant"

search = search.lower()
cur.execute(f"""
    select e.id, a.title, a.url, e.vector
    from embeddings e
    join articles a
        using(id)
    where lower(a.title) like "%{search}%"
    order by random()
    limit 5
""")
for row in cur.fetchall():
    gid, gtitle, gurl, gembed = row
    print('\n' + gtitle)
    print('\n', gid)
    print(
        distance.cosine(
            json.loads(gembed),
            json.loads(tembed)
        )
    )


Chandrakant Bakshi

 4838055
0.9770822469649769

Chandrakanta

 4485954
0.9118056200772688

Taito Kantonen

 4674735
0.8692198498418106

Srikanth Deva

 4523451
1.0307617706633563

Ravikant Shukla

 4059578
1.0068879967279543


In [51]:
from scipy.spatial import distance
import json

distance.cosine(
    json.loads(gembed),
    json.loads(tembed)
)

0.9474401633002533